# Step 3 ES Features Market Level 
-  from eS_monthly, make market level variables
- Use es_labeled90_100_2two_step.csv - clustering that only clustered ES data points 
-  Last Edit: June 25, 2020 

In [1]:
import pandas as pd

# input:  the clustering results with specific parameters - market centroid coordinates and labels ( market numbering)

In [3]:
 
temp_labels=pd.read_csv('../2_pipeline/es_labeled90_100_2two_step.csv')

In [4]:
temp_labels.tail()

,Unnamed: 0,labels,lat,long
458,139,36,42.519966,-71.763321
459,140,36,42.564663,-71.567299
460,141,17,42.671616,-73.246979
461,142,17,42.379925,-72.604042
462,143,17,42.364033,-72.458755


# Input:  import the coord data with installer ids that need to be labeled with their cluster label 

In [7]:
total_df_url='../0_data/dec14_total_df.csv' 
total_df=pd.read_csv(total_df_url)
total_df.head()

,Unnamed: 0,installerid,lat,long,isES
0,1,205,45.569080,-122.946701,1
1,2,216,42.503670,-83.781059,1
2,4,20831,34.004478,-117.684700,1
3,5,363,33.610741,-111.891472,1
4,6,21695,29.985395,-95.346214,1


# Merge installer id with market label label data 

In [8]:

labeled_totaldf=pd.merge(total_df[total_df['isES']==1][['installerid','lat','long']], temp_labels[['labels','lat','long']], how='left', on=['lat', 'long'])
labeled_totaldf=labeled_totaldf.dropna()
labeled_totaldf.head()

,installerid,lat,long,labels
0,205,45.569080,-122.946701,0
1,216,42.503670,-83.781059,14
2,20831,34.004478,-117.684700,4
3,363,33.610741,-111.891472,6
4,21695,29.985395,-95.346214,30


In [9]:
labeled_totaldf.describe()

,lat,long,labels
count,530.000000,530.000000,530.000000
mean,37.702916,-94.247910,16.384906
std,4.824161,19.774081,13.304595
min,25.485147,-122.987328,-1.000000
25%,33.987985,-117.257345,4.000000
50%,38.804840,-86.463371,17.000000
75%,41.595521,-75.035137,30.750000
max,48.796490,-69.747612,36.000000


In [10]:
set(labeled_totaldf.labels)

{-1,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36}

# Input:  Import the individual level ES monthly data

Note: with avg_sent_score

In [11]:
es_monthly_ind=pd.read_stata('../2_pipeline/es_monthly_ind.dta')
es_monthly_ind.tail(5)

,five,four,three,two,one,avg_sent_score,installer_id,quotes_count,winning_quotes_count,year_month_count,reviews_ct,avg
8650,6,0,0,0,0,0.56655,22992,10.0,0.0,64.0,6.0,5.0
8651,0,0,0,0,0,NaN,22997,28.0,0.0,63.0,0.0,NaN
8652,2,0,0,0,0,0.65500,22997,4.0,0.0,64.0,2.0,5.0
8653,0,0,0,0,0,NaN,23027,2.0,0.0,63.0,0.0,NaN
8654,3,0,0,0,0,0.69600,23027,15.0,0.0,64.0,3.0,5.0


# convert ES monthly data to include platform market level data 

## set the helper functions 

In [12]:
#  helper functions 
def find_es_installers_id(marketid,labeled_totaldf):
    esids=labeled_totaldf[labeled_totaldf.labels==marketid].installerid
    return esids
#overwrite previous find_es_istallers_id 


def find_es_monthly(esids,esmonthly):
    if(len(esids)>0):
        return esmonthly.loc[esmonthly['installer_id'].isin(esids)]
    else :
        return

 

## main steps - create the ES panel 

In [13]:
es_panel=pd.DataFrame(columns=es_monthly_ind.columns)
for market in set(temp_labels.labels.unique()):
    #print(market)
    #find_es_monthly(find_es_installers_id(market,labeled_totaldf),es_monthlyperf)
    df=find_es_monthly(find_es_installers_id(market,labeled_totaldf),es_monthly_ind)
    try:
        df.loc[:,'market']=market
        es_panel=es_panel.append(df,ignore_index=True)
    except:
        print('market '+str(market)+' has an exception..not a big deal. Probably just a lack of ES installers')

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Anaconda\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and

market 19 has an exception..not a big deal. Probably just a lack of ES installers


In [14]:
es_panel.tail()

,avg,avg_sent_score,five,four,installer_id,market,one,quotes_count,reviews_ct,three,two,winning_quotes_count,year_month_count
8108,5.0,0.413850,2,0,22790,-1.0,0,2.0,2.0,0,0,0.0,60.0
8109,5.0,0.413850,2,0,22790,-1.0,0,7.0,2.0,0,0,0.0,61.0
8110,5.0,0.413850,2,0,22790,-1.0,0,11.0,2.0,0,0,0.0,62.0
8111,5.0,0.619325,4,0,22790,-1.0,0,4.0,4.0,0,0,0.0,63.0
8112,5.0,0.619325,4,0,22790,-1.0,0,2.0,4.0,0,0,0.0,64.0


In [15]:
es_panel.market.unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., -1.])

In [16]:
es_panel.describe()

,avg,avg_sent_score,market,quotes_count,reviews_ct,winning_quotes_count,year_month_count
count,4953.000000,4953.000000,8113.000000,6031.000000,8113.000000,6031.000000,8113.000000
mean,4.880355,0.683348,17.784667,39.029348,3.397387,0.645001,43.258228
std,0.397170,0.163028,13.486967,94.105720,5.802378,1.675744,14.543704
min,1.000000,-0.826800,-1.000000,1.000000,0.000000,0.000000,1.000000
25%,5.000000,0.634175,4.000000,3.000000,0.000000,0.000000,34.000000
50%,5.000000,0.704000,17.000000,11.000000,1.000000,0.000000,46.000000
75%,5.000000,0.777200,33.000000,34.000000,4.000000,1.000000,55.000000
max,5.000000,0.957600,36.000000,1417.000000,52.000000,43.000000,64.000000


## handling missing values 

- it may not be very fair to fill avg nan to 0. We can come back to check this later 

# make market level variables for ES

Things need to compute for  group level(group including self or excluding self):  
-  avgmean
- avg_sent_score


** Naming (excluding self) 
- avgmean_others
- avg_sent_score_others


## prepping columns for dataframe structure

In [17]:

columns_names=['year_month_count','market','installer_id','avgmean_mkt','avgmean_others','avg_sent_score_mkt','avg_sent_score_others']

In [18]:
es_grouped=pd.DataFrame(data=[],columns=columns_names)

In [19]:
es_grouped

,year_month_count,market,installer_id,avgmean_mkt,avgmean_others,avg_sent_score_mkt,avg_sent_score_others


In [20]:
es_panel.tail()

,avg,avg_sent_score,five,four,installer_id,market,one,quotes_count,reviews_ct,three,two,winning_quotes_count,year_month_count
8108,5.0,0.413850,2,0,22790,-1.0,0,2.0,2.0,0,0,0.0,60.0
8109,5.0,0.413850,2,0,22790,-1.0,0,7.0,2.0,0,0,0.0,61.0
8110,5.0,0.413850,2,0,22790,-1.0,0,11.0,2.0,0,0,0.0,62.0
8111,5.0,0.619325,4,0,22790,-1.0,0,4.0,4.0,0,0,0.0,63.0
8112,5.0,0.619325,4,0,22790,-1.0,0,2.0,4.0,0,0,0.0,64.0


## Looping over month, then over market , to create market_level_data and put it to dataframe es_grouped 
- for a given month, a given market
- es_panel contains all the installer level records 
- working_df finds the set of es_panel records on the market for that given month 
- working_df_not_self exclude self
- working_df_w_self include self 
- avgmean_mkt: market average rating. Include self. (useful for market level analysis) 
- avg_sent_score_mkt: market average sentiment. Include self(useful for market level analysis) 
- avgmean_others: average rating of others 
- avg_sent_score_others: average sentiment score or others 
- market_level_data: collect the aforementioned variables 

In [21]:
datalist=[]
for month in set(es_panel.year_month_count):
    print('month: '+str(month))
    temp_working_df=es_panel[es_panel['year_month_count']==month]
    for market in set(temp_working_df.market):
        print('market'+str(market))
        working_df=temp_working_df[temp_working_df['market']==market]
        for i in set(working_df.installer_id):
            working_df_not_self=working_df[working_df['installer_id']!=i]
            working_df_w_self=working_df
            avgmean_mkt=working_df_w_self.avg.mean()
            avg_sent_score_mkt=working_df_w_self.avg_sent_score.mean()
            avgmean_others=working_df_not_self.avg.mean()
            avg_sent_score_others=working_df_not_self.avg_sent_score.mean()
            market_level_data=[month,market,i,avgmean_mkt,avgmean_others,avg_sent_score_mkt,avg_sent_score_others]
            datalist.append(market_level_data)

month: 1.0
market36.0
market-1.0
month: 2.0
market33.0
market34.0
market35.0
market36.0
market-1.0
month: 3.0
market33.0
market34.0
market35.0
market36.0
market17.0
market-1.0
month: 4.0
market33.0
market34.0
market35.0
market36.0
market17.0
market20.0
market-1.0
month: 5.0
market33.0
market34.0
market35.0
market36.0
market17.0
market20.0
market-1.0
month: 6.0
market33.0
market34.0
market35.0
market36.0
market9.0
market17.0
market20.0
market-1.0
month: 7.0
market1.0
market2.0
market33.0
market34.0
market35.0
market36.0
market9.0
market17.0
market20.0
market21.0
market-1.0
month: 8.0
market1.0
market2.0
market33.0
market4.0
market34.0
market35.0
market36.0
market9.0
market17.0
market20.0
market21.0
market-1.0
market30.0
month: 9.0
market1.0
market2.0
market33.0
market4.0
market34.0
market6.0
market35.0
market36.0
market9.0
market11.0
market17.0
market20.0
market21.0
market-1.0
market30.0
month: 10.0
market1.0
market2.0
market33.0
market4.0
market34.0
market6.0
market35.0
market36.0
mark

In [22]:
datalist[-20:-15]

[[64.0,
  36.0,
  22992,
  4.920098304748535,
  4.915104389190674,
  0.6961315870285034,
  0.7042304277420044],
 [64.0,
  36.0,
  21457,
  4.920098304748535,
  4.915104389190674,
  0.6961315870285034,
  0.6889023184776306],
 [64.0,
  36.0,
  21363,
  4.920098304748535,
  4.920312881469727,
  0.6961315870285034,
  0.7014122009277344],
 [64.0,
  36.0,
  20629,
  4.920098304748535,
  4.915104389190674,
  0.6961315870285034,
  0.6983884572982788],
 [64.0,
  36.0,
  21143,
  4.920098304748535,
  4.922246932983398,
  0.6961315870285034,
  0.6964492797851562]]

In [23]:
es_grouped=pd.DataFrame(data=datalist,columns=columns_names)

In [24]:
es_grouped.describe()

,year_month_count,market,installer_id,avgmean_mkt,avgmean_others,avg_sent_score_mkt,avg_sent_score_others
count,8113.000000,8113.000000,8113.000000,6984.000000,6634.000000,6984.000000,6634.000000
mean,43.258228,17.784667,17971.731912,4.891938,4.891381,0.693654,0.692523
std,14.543701,13.486967,6707.006717,0.183281,0.198480,0.077837,0.088881
min,1.000000,-1.000000,108.000000,3.000000,1.000000,0.059340,-0.599400
25%,34.000000,4.000000,20131.000000,4.874211,4.872396,0.663772,0.662121
50%,46.000000,17.000000,20627.000000,4.944445,4.944445,0.698096,0.698012
75%,55.000000,33.000000,21350.000000,5.000000,5.000000,0.735656,0.734887
max,64.000000,36.000000,23027.000000,5.000000,5.000000,0.886075,0.957600


In [25]:
es_panel.head()

,avg,avg_sent_score,five,four,installer_id,market,one,quotes_count,reviews_ct,three,two,winning_quotes_count,year_month_count
0,NaN,NaN,0,0,21283,0.0,0,2.0,0.0,0,0,0.0,35.0
1,NaN,NaN,0,0,21283,0.0,0,4.0,0.0,0,0,0.0,36.0
2,NaN,NaN,0,0,21283,0.0,0,4.0,0.0,0,0,0.0,37.0
3,NaN,NaN,0,0,21283,0.0,0,3.0,0.0,0,0,0.0,38.0
4,NaN,NaN,0,0,21283,0.0,0,7.0,0.0,0,0,0.0,39.0


## Part 1 of MKT level variables: avgmean, avgstd, reviews_c on mkt or others level 
merge es_grouped and es_panel 

In [26]:
es_merged_mkt_p1=es_panel.merge(es_grouped,on=['year_month_count','market','installer_id'],how='inner')

In [27]:
es_merged_mkt_p1.tail()

,avg,avg_sent_score,five,four,installer_id,market,one,quotes_count,reviews_ct,three,two,winning_quotes_count,year_month_count,avgmean_mkt,avgmean_others,avg_sent_score_mkt,avg_sent_score_others
8108,5.0,0.413850,2,0,22790,-1.0,0,2.0,2.0,0,0,0.0,60.0,4.856640,4.845613,0.673715,0.693705
8109,5.0,0.413850,2,0,22790,-1.0,0,7.0,2.0,0,0,0.0,61.0,4.735479,4.716584,0.580091,0.591965
8110,5.0,0.413850,2,0,22790,-1.0,0,11.0,2.0,0,0,0.0,62.0,4.619199,4.593812,0.564844,0.574910
8111,5.0,0.619325,4,0,22790,-1.0,0,4.0,4.0,0,0,0.0,63.0,4.602383,4.577532,0.571987,0.569029
8112,5.0,0.619325,4,0,22790,-1.0,0,2.0,4.0,0,0,0.0,64.0,4.440794,4.384874,0.513098,0.502476


Part 2 of MKT level variables 

## Part 2 of MKT level variables: entropy related variables 

### Part 2 Helper function: Entropy Calculation Function



In [28]:
import pandas as pd
import scipy.stats

def entropy(data):
    """Calculates entropy of the passed `pd.Series`
    """
    entropy=0
    if len(data)>0:
        p_data = data.value_counts()           # counts occurrence of each value
        entropy = scipy.stats.entropy(p_data)  # get entropy from counts
    return entropy

###  Part 2 helper data list 'es_market_ids' 

In [29]:
es_merged_mkt_p1.describe()

,avg,avg_sent_score,market,quotes_count,reviews_ct,winning_quotes_count,year_month_count,avgmean_mkt,avgmean_others,avg_sent_score_mkt,avg_sent_score_others
count,4953.000000,4953.000000,8113.000000,6031.000000,8113.000000,6031.000000,8113.000000,6984.000000,6634.000000,6984.000000,6634.000000
mean,4.880355,0.683348,17.784667,39.029348,3.397387,0.645001,43.258228,4.891938,4.891381,0.693654,0.692523
std,0.397170,0.163028,13.486967,94.105720,5.802378,1.675744,14.543704,0.183281,0.198480,0.077837,0.088881
min,1.000000,-0.826800,-1.000000,1.000000,0.000000,0.000000,1.000000,3.000000,1.000000,0.059340,-0.599400
25%,5.000000,0.634175,4.000000,3.000000,0.000000,0.000000,34.000000,4.874211,4.872396,0.663772,0.662121
50%,5.000000,0.704000,17.000000,11.000000,1.000000,0.000000,46.000000,4.944445,4.944445,0.698096,0.698012
75%,5.000000,0.777200,33.000000,34.000000,4.000000,1.000000,55.000000,5.000000,5.000000,0.735656,0.734887
max,5.000000,0.957600,36.000000,1417.000000,52.000000,43.000000,64.000000,5.000000,5.000000,0.886075,0.957600


In [30]:
es_market_ids=es_merged_mkt_p1[['installer_id','year_month_count','market']]

## Input: import the  raw review data /0_data/Lock_ES_RawData/installer_review_data_20180410.csv' 

In [32]:
reviews=pd.read_csv('../0_data/Lock_ES_RawData/installer_review_data_20180410.csv',sep=',',escapechar='\\')
reviews=reviews[['installer_id','date_created','overall_rating']]

reviews['date_created']=pd.to_datetime(reviews['date_created'])

In [33]:
reviews['year']=reviews['date_created'].apply(lambda x:x.year)
reviews['month']=reviews['date_created'].apply(lambda x:x.month)
reviews['year_month_count']=12*(reviews['year']-2013)+reviews['month']

In [34]:
reviews.tail()

,installer_id,date_created,overall_rating,year,month,year_month_count
3383,3247,2018-04-09 15:34:12,1,2018,4,64
3384,21242,2018-04-09 16:05:47,5,2018,4,64
3385,21269,2018-04-09 19:49:58,5,2018,4,64
3386,21711,2018-04-09 21:57:33,5,2018,4,64
3387,21866,2018-04-10 19:04:47,4,2018,4,64


In [35]:
reviews['installer_id']=reviews['installer_id'].astype(int)

## Entropy calculation
- entropy of reviews ratings on the entire market 
    - name: ent_mkt
    - note: it is not the most efficient place to calculate market level entropy because we are repeating a lot of calculations. 
- entropy of reviews ratings on the market excluding self 
    - name:ent_others 

In [37]:
result_list=[]
for month in set(es_market_ids.year_month_count):
    temp1=es_market_ids[es_market_ids['year_month_count']==month]
    for market in set(temp1.market):
        temp2=temp1[temp1['market']==market]
        installer_id_list=temp2.installer_id.tolist()
        relevant_reviews=reviews[reviews['year_month_count']<=month]
        relevant_reviews=relevant_reviews[relevant_reviews['installer_id'].isin(installer_id_list)]
        if len(relevant_reviews)>0:
        #print(relevant_reviews.overall_rating)
            #ent=entropy(relevant_reviews['overall_rating'].to_numpy())
            print(len(relevant_reviews))
            temp=relevant_reviews['overall_rating']
            ent_mkt=entropy(temp)
            #print('the ent_mkt is {}'.format(ent_mkt))
            for installer in set(relevant_reviews.installer_id):
                temp_others=relevant_reviews[relevant_reviews.installer_id!=installer].overall_rating
                ent_others=entropy(temp_others)
                temp_self=relevant_reviews[relevant_reviews.installer_id==installer].overall_rating
                ent_self=entropy(temp_self)
                #print('the ent_others is {}'.format(ent_others))
                data_collect=[month,market,installer,ent_mkt,ent_others,ent_self]
                print(data_collect)
                result_list.append(data_collect)
            
        
        

2
[7.0, 34.0, 857, 0.0, 0, 0.0]
1
[7.0, 36.0, 6507, 0.0, 0, 0.0]
2
[8.0, 34.0, 857, 0.0, 0, 0.0]
3
[8.0, 36.0, 554, 0.0, 0.0, 0.0]
[8.0, 36.0, 6507, 0.0, 0.0, 0.0]
2
[9.0, 34.0, 857, 0.0, 0, 0.0]
3
[9.0, 36.0, 554, 0.0, 0.0, 0.0]
[9.0, 36.0, 6507, 0.0, 0.0, 0.0]
2
[10.0, 34.0, 857, 0.0, 0, 0.0]
3
[10.0, 36.0, 554, 0.0, 0.0, 0.0]
[10.0, 36.0, 6507, 0.0, 0.0, 0.0]
2
[11.0, 34.0, 857, 0.0, 0, 0.0]
3
[11.0, 36.0, 554, 0.0, 0.0, 0.0]
[11.0, 36.0, 6507, 0.0, 0.0, 0.0]
2
[12.0, 34.0, 857, 0.0, 0, 0.0]
3
[12.0, 36.0, 554, 0.0, 0.0, 0.0]
[12.0, 36.0, 6507, 0.0, 0.0, 0.0]
2
[13.0, 34.0, 857, 0.0, 0, 0.0]
3
[13.0, 36.0, 554, 0.0, 0.0, 0.0]
[13.0, 36.0, 6507, 0.0, 0.0, 0.0]
1
[14.0, 31.0, 6515, 0.0, 0, 0.0]
2
[14.0, 34.0, 857, 0.0, 0, 0.0]
4
[14.0, 36.0, 554, 0.0, 0.0, 0.0]
[14.0, 36.0, 6507, 0.0, 0.0, 0.0]
[14.0, 36.0, 6356, 0.0, 0.0, 0.0]
2
[15.0, 31.0, 6515, 0.0, 0, 0.0]
2
[15.0, 34.0, 857, 0.0, 0, 0.0]
4
[15.0, 36.0, 554, 0.0, 0.0, 0.0]
[15.0, 36.0, 6507, 0.0, 0.0, 0.0]
[15.0, 36.0, 6356, 0.0,

In [38]:
len(result_list)

5340

In [39]:
type(result_list[:3])

list

In [40]:
col_names=['year_month_count','market','installer_id','ent_mkt','ent_others','ent_self']

In [41]:
es_merged_mkt_p2_entropy=pd.DataFrame(data=result_list,columns=col_names)

In [42]:
es_merged_mkt_p2_entropy.tail()

,year_month_count,market,installer_id,ent_mkt,ent_others,ent_self
5335,64.0,-1.0,21386,0.708778,0.599394,0.562335
5336,64.0,-1.0,21455,0.708778,0.722076,0.000000
5337,64.0,-1.0,566,0.708778,0.727580,0.578325
5338,64.0,-1.0,21175,0.708778,0.722076,0.000000
5339,64.0,-1.0,21534,0.708778,0.757809,0.000000


## merge market level variables with entropy variables

### 1. Part 1 market level variables

In [43]:
es_merged_mkt_p1.describe()

,avg,avg_sent_score,market,quotes_count,reviews_ct,winning_quotes_count,year_month_count,avgmean_mkt,avgmean_others,avg_sent_score_mkt,avg_sent_score_others
count,4953.000000,4953.000000,8113.000000,6031.000000,8113.000000,6031.000000,8113.000000,6984.000000,6634.000000,6984.000000,6634.000000
mean,4.880355,0.683348,17.784667,39.029348,3.397387,0.645001,43.258228,4.891938,4.891381,0.693654,0.692523
std,0.397170,0.163028,13.486967,94.105720,5.802378,1.675744,14.543704,0.183281,0.198480,0.077837,0.088881
min,1.000000,-0.826800,-1.000000,1.000000,0.000000,0.000000,1.000000,3.000000,1.000000,0.059340,-0.599400
25%,5.000000,0.634175,4.000000,3.000000,0.000000,0.000000,34.000000,4.874211,4.872396,0.663772,0.662121
50%,5.000000,0.704000,17.000000,11.000000,1.000000,0.000000,46.000000,4.944445,4.944445,0.698096,0.698012
75%,5.000000,0.777200,33.000000,34.000000,4.000000,1.000000,55.000000,5.000000,5.000000,0.735656,0.734887
max,5.000000,0.957600,36.000000,1417.000000,52.000000,43.000000,64.000000,5.000000,5.000000,0.886075,0.957600


### 2. entropy variables 

In [44]:
es_merged_mkt_p2_entropy.tail()

,year_month_count,market,installer_id,ent_mkt,ent_others,ent_self
5335,64.0,-1.0,21386,0.708778,0.599394,0.562335
5336,64.0,-1.0,21455,0.708778,0.722076,0.000000
5337,64.0,-1.0,566,0.708778,0.727580,0.578325
5338,64.0,-1.0,21175,0.708778,0.722076,0.000000
5339,64.0,-1.0,21534,0.708778,0.757809,0.000000


In [45]:
es_merged=es_merged_mkt_p1.merge(es_merged_mkt_p2_entropy,on=['year_month_count','market','installer_id'],how='outer')

In [46]:
es_merged.describe()

,avg,avg_sent_score,market,quotes_count,reviews_ct,winning_quotes_count,year_month_count,avgmean_mkt,avgmean_others,avg_sent_score_mkt,avg_sent_score_others,ent_mkt,ent_others,ent_self
count,4953.000000,4953.000000,8113.000000,6031.000000,8113.000000,6031.000000,8113.000000,6984.000000,6634.000000,6984.000000,6634.000000,5340.000000,5340.000000,5340.000000
mean,4.880355,0.683348,17.784667,39.029348,3.397387,0.645001,43.258228,4.891938,4.891381,0.693654,0.692523,0.237623,0.224737,0.098143
std,0.397170,0.163028,13.486967,94.105720,5.802378,1.675744,14.543704,0.183281,0.198480,0.077837,0.088881,0.185722,0.191554,0.219973
min,1.000000,-0.826800,-1.000000,1.000000,0.000000,0.000000,1.000000,3.000000,1.000000,0.059340,-0.599400,0.000000,0.000000,0.000000
25%,5.000000,0.634175,4.000000,3.000000,0.000000,0.000000,34.000000,4.874211,4.872396,0.663772,0.662121,0.139061,0.000000,0.000000
50%,5.000000,0.704000,17.000000,11.000000,1.000000,0.000000,46.000000,4.944445,4.944445,0.698096,0.698012,0.237177,0.224707,0.000000
75%,5.000000,0.777200,33.000000,34.000000,4.000000,1.000000,55.000000,5.000000,5.000000,0.735656,0.734887,0.314123,0.314945,0.000000
max,5.000000,0.957600,36.000000,1417.000000,52.000000,43.000000,64.000000,5.000000,5.000000,0.886075,0.957600,1.054920,1.070593,1.209574


## Output - es_marketlevel data 
- note: it's called marketlevel, but it's not organized at marketlevel. It's just installer level plus market level variables 

In [47]:
parameters='90_100_2twostep'

In [48]:
es_merged.to_csv('../2_pipeline/es_marketlevel'+parameters+'.csv')